In [1]:
import psutil
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
import gc
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#load data
data = pd.read_csv('../input/practice-log-demographics/Practice_Log_Demographics (1).csv', low_memory=False)

In [3]:
#the features that would be used
FEATURES = ['chapter_label', 'sub_chapter_label','question_name','user_id.x','term','STDNT_SEX_CD', 
            'NonNativeEnglish', 'White', 'Asian','WhiteOrAsian', 'Hispanic','AfricanAmerican','OtherEthnicities',
            'NonWhiteOrAsian','STDNT_CTZN_STAT_CD', 'international','gradingType','birthYear','exclClassCumGPA',
            'Freshman', 'Junior','Sophomore','Senior','termCreditsGPA','termCreditsNoGPA','athlete_1','honorsPro',
            'LSA', 'programBusiness', 'programEngineering','programInformation', 'programOther','HSCalculusTaken', 
            'highSchoolGPA', 'majorsCount', 'minorsCount','PREV_TERM_CUM_GPA', 'classGraded', 'classHonors', 
            'Pass_Fail', 'parentsGraduateEdu',  'minorityGroup', 'q',"day's_available_flashcards", 
            'start_practice', 'end_practice','days_offset']

In [4]:
#use only the columns that are in FEATURES
data = data[FEATURES]

In [5]:
data['term'] = data['term'].astype('category')

In [6]:
gc.collect()

80

In [7]:
#one hot encode COVID_Graded feature
data['COVID_Graded'] = np.where(data['gradingType']=='COVID Graded', 1, 0)
data.drop(columns=['gradingType'], inplace=True)

In [8]:
#create age column where the age of the user would be included
data['term_value'] = [int(ele[3:]) for ele in data['term']]
data['age'] = data['term_value'] - data['birthYear']

In [9]:
data.drop(columns=['term_value', 'birthYear'], inplace=True)

In [10]:
#convert start_practice and end_practice to datatime column
data.start_practice = pd.to_datetime(data.start_practice, format='%Y-%m-%d %H:%M:%S')
data.end_practice = pd.to_datetime(data.end_practice, format='%Y-%m-%d %H:%M:%S')
#label encode term, chapter_label, question_name, sub_chapter_label columns
data['chapter_id'] = data['chapter_label'].replace(data['chapter_label'].unique(), list(range(data['chapter_label'].nunique()))) + 1
data['sub_chapter_id'] = data['sub_chapter_label'].replace(data['sub_chapter_label'].unique(), list(range(data['sub_chapter_label'].nunique()))) + 1
data['question_id'] = data['question_name'].replace(data['question_name'].unique(), list(range(data['question_name'].nunique()))) + 1

In [11]:
data['dif'] = data.end_practice - data.start_practice
data['dif'] = data['dif'] /np.timedelta64(1, 's')
data['q'] = np.where((data['q']==5) & (data['dif'] <= 60), 1, 0)

In [12]:
#drop chapter_label, sub_chapter_label, question_name
data.drop(columns = ['chapter_label', 'sub_chapter_label', 'question_name'], inplace=True)

In [13]:
#transform minor_count column
new_minors_count = []
for i in data['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

data['minorsCount'] = new_minors_count

In [14]:
data['term']#sort data based on timestamp
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)

In [15]:
#convert user_id column to the int type user_id column
data['user_id.x'] = data['term'].astype(str) + data['user_id.x'].astype(str)
data['user_id.x'] = data['user_id.x'].replace(data['user_id.x'].unique(), list(range(data['user_id.x'].nunique())))

In [16]:
data['term'] =  data['term'].cat.codes

In [17]:
data['prev_time_elapsed'] = None
data['time_lag'] = None
data['time_lag'] = data['time_lag'].astype(np.float)
data['prev_time_elapsed'] = data['prev_time_elapsed'].astype(np.float)

In [18]:
unique_chapts = data['chapter_id'].unique()
from tqdm import tqdm
cnt = 0
for user in tqdm(data['user_id.x'].unique()):
        for chapter in unique_chapts:
            tmp_user = data[(data['user_id.x']==user) & (data['chapter_id']==chapter)]
            #if user doesn't answer any question from the chapter
            if len(tmp_user) < 1:
                continue
            #calculate average elapsed time before the current question
            #from its corresponding chapter
            tmp_time_elapsed = tmp_user.end_practice - tmp_user.start_practice
            tmp_time_elapsed = tmp_time_elapsed / np.timedelta64(1, 's')
            tmp_time_elapsed = np.insert(np.array(tmp_time_elapsed[:-1]), 0, 0., axis=0)
            tmp_time_elapsed = np.cumsum(tmp_time_elapsed)
            indices = tmp_user.index
            start_row = indices[0]
            data['time_lag'].iloc[start_row] = 0
            for_mean = np.arange(len(tmp_user))
            for_mean[0] = 1
            time_substrahend = tmp_user.start_practice.iloc[:-1]
            time_substrahend = time_substrahend.apply(lambda a: a.timestamp())
            time_substrahend = np.array(time_substrahend)
            
            time_minuend = tmp_user.start_practice.iloc[1:]
            time_minuend = time_minuend.apply(lambda a: a.timestamp())
            time_minuend = np.array(time_minuend)

            data['prev_time_elapsed'].iloc[indices] = tmp_time_elapsed/for_mean
            #calculate difference between current and last interaction
            #from the same chapter
            data['time_lag'].iloc[indices[1:]] = time_minuend - time_substrahend

100%|██████████| 1110/1110 [07:59<00:00,  2.31it/s]


In [19]:
#now we can drop end practice
data.drop(columns=['end_practice'], inplace=True)
# data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)
#replace column q with anwered_correctly
data['answered_correctly'] = data['q']
data.drop(columns='q', inplace=True)
gc.collect()

46

In [20]:
gc.collect()

20

In [21]:
data.fillna(0, inplace=True)

In [22]:
#set maximum sequence length to 400
#n_part to the number of unique sub_chapter_id
#dimension of the embedding is 128
#number of encoder and decoders are 2
#and the dropout is set to 0.2
n_skill = data['question_id'].nunique() + 1

In [23]:
#create dataset class
#to prepare it for train and valid sets
class PRACTICE_DATASET(Dataset):
    def __init__(self, data, maxlength=400, test=False, n_skill=n_skill):
        super(PRACTICE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.test = test
        self.users = list()
        self.n_skill = n_skill
        for user in data.index:
            self.users.append(user)
            
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        question_id, chapter, schapter, term, user_feats, labels = self.data[user]
        question_id = np.array(question_id, np.int16)
        chapter = np.array(chapter, np.int16)
        schapter = np.array(schapter, np.int16)
        user_feats = np.array(user_feats, np.float)
        labels = np.array(labels, np.int8)
        
        q = np.zeros(self.maxlength, dtype=int)
        qa = np.zeros(self.maxlength, dtype=int)
        
        term = term.values
        term_encode = [0]*7
        term_encode[term[0]] = 1
        n = len(question_id)
#         print(user_feats.shape)
        #if more the user interactions are more than self.maxlength (400)
        #only the last self.maxlength interactions will be included
        #if the number of user interactions would be less than self.maxlength
        #the padding would be utilized        
        if n > self.maxlength:
            question_id = question_id[-self.maxlength : ]
            chapter = chapter[-self.maxlength :]
            schapter = schapter[-self.maxlength: ]
            user_feats = user_feats[-self.maxlength :, :]
            labels = labels[-self.maxlength: ]
            responses = np.append(2, labels[:-1])
            q[-self.maxlength:] = question_id
            qa[-self.maxlength:] = labels
        else:
            q[-n:] = question_id
            qa[-n:] = labels
            question_id = np.pad(question_id, (self.maxlength - n, 0))
            chapter = np.pad(chapter, (self.maxlength - n, 0))
            schapter = np.pad(schapter, (self.maxlength - n, 0))
            user_feats = np.pad(user_feats, pad_width=((0, self.maxlength - n), (0, 0)))
            user_feats = np.array(user_feats, np.float)
            responses = np.append(2, labels[:-1])
            labels = np.pad(labels, (self.maxlength - n, 0))
            responses = np.pad(responses, (self.maxlength - n, 0), mode='constant', constant_values = 2)
            
#         print(np.array(term).shape)
#         term = [term] * self.maxlength
#         print(np.array(term).shape)
#         user_feats = np.hstack([user_feats, term])

        target_id = q[1:]
        label = qa[1:]
        
        new_features = []
        count = 0
        for f in user_feats:
          temp = list(f)
          temp.extend(term_encode)
          new_features.append(temp)
          count += 1
        user_feats = new_features
        x = np.zeros(self.maxlength-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill
    
        return x, target_id, chapter[1:], schapter[1:], responses[1:], np.array(user_feats[1:]), labels[1:] 


In [24]:
#normalize numeric features
NUMERIC_FEATURE =  ['age',
            'exclClassCumGPA',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            "day's_available_flashcards", 
            'days_offset', 
            'prev_time_elapsed',
             'time_lag']
 
for f in NUMERIC_FEATURE:
  m = data[f].mean()
  std = data[f].std()
  data[f] = (data[f] - m)/std

In [25]:
#group data based on user id
group = data.groupby(['user_id.x']).apply(lambda r: (
                r['question_id'],
                r['chapter_id'],
                r['sub_chapter_id'],
                r['term'],
                np.array([
                    r['STDNT_SEX_CD'],r['STDNT_CTZN_STAT_CD'], r['international'], 
                  r['White'],r['Asian'],r['NonWhiteOrAsian'], r['age'],r['exclClassCumGPA'],
                r['Freshman'], r['Junior'], r['Sophomore'], r['Senior'],
                r['termCreditsGPA'], r['termCreditsNoGPA'],
                r['athlete_1'], r['honorsPro'],
                r['LSA'], r['programBusiness'], r['programEngineering'], 
                r['programInformation'], r['programOther'],
                r['HSCalculusTaken'],  r['highSchoolGPA'], 
                r['majorsCount'], r['minorsCount'],
                r['PREV_TERM_CUM_GPA'], 
                r['parentsGraduateEdu'], r['minorityGroup'],
                r["day's_available_flashcards"],
                r['days_offset'],
#                 r['prev_time_elapsed'],
#                 r['time_lag']
              ]).transpose(),
                r['answered_correctly'],
                ))

In [26]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in group.index:
  if len(group[index][0]) <= 10:
    toRemove.append(index)
group = group.drop(index=toRemove)

In [27]:
n_part = data['sub_chapter_id'].nunique() + 1

In [28]:
#feed forward network
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

#define mask that would be used in multi head attention layer
def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

#define the SAKT model
class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=600, embed_dim=128, dropout_rate=0.2):
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim
        self.n_chapter= 39
        self.n_sub_chapter = n_part
        self.max_seq = max_seq
        
        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(600+1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim) ##question categorical embedding
        self.c_embedding = nn.Embedding(self.n_chapter+1, embed_dim) ## chapter categorical embedding
        self.sc_embedding = nn.Embedding(self.n_sub_chapter, embed_dim) ## subchapter categorical embedding
        self.res_embedding = nn.Embedding(2+1, embed_dim) ## response (answered_correctly) categorical embedding
        self.feat_embedding = nn.Linear(37, embed_dim)##student features linear embedding
        
        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=dropout_rate)

        self.dropout = nn.Dropout(dropout_rate)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids, chapter, schapter,response, user_features):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
    
        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x
        ## embedding layers
        e = self.e_embedding(question_ids)
        chapter = self.c_embedding(chapter)
        schapter = self.sc_embedding(schapter)
        res = self.res_embedding(response)
        user_features = self.feat_embedding(user_features)
        
        x = x  + user_features + chapter + schapter + res
        
        x = x.permute(1, 0, 2) 
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) 

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

In [29]:
#in case the validation loss would 
#not improve for $patienc$ consecutive 
#epochs the training would be terminated
patience = 5

In [30]:
#use KFold
X = np.array(group.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
train_accs = list()
val_losses = list()
val_aucs = list()
val_accs = list()
test_losses = list()
test_aucs = list()
test_accs = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train = PRACTICE_DATASET(group[users_train])
    valid = PRACTICE_DATASET(group[users_val])
    test = PRACTICE_DATASET(group[users_test])
    train_dataloader = DataLoader(train, batch_size=64, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(valid, batch_size=64, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=8)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    saint = SAKTModel(n_skill, n_part)
    epochs = 100
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(saint.parameters(), betas=(0.9, 0.999), lr = 0.0005, eps=1e-8)
    saint.to(device)
    criterion.to(device)
    
    def train_epoch(model=saint, train_iterator=train_dataloader, optim=optimizer, criterion=criterion, device=device):
        model.train()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(train_iterator)
        for item in tbar:
            x = item[0].to(device).long()
            question_id = item[1].to(device).long()
            chapter = item[2].to(device).long()
            schapter = item[3].to(device).long()
            responses = item[4].to(device).long()
#             user_feats = np.array(item[5])
            user_feats = item[5].to(device).float()
            label = item[6].to(device).float()            
            target_mask = (question_id!=0)
            optim.zero_grad()
            output = model(x, question_id, chapter, schapter, responses, user_feats)
            output = torch.reshape(output, label.shape)

            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            pred = (torch.sigmoid(output) >= 0.5).long()

            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('loss - {:.4f}'.format(loss))
        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.mean(train_loss)

        return loss, acc, auc
   

    def val_epoch(model=saint, val_iterator=test_dataloader, 
              criterion=criterion, device=device):
        model.eval()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(val_iterator)
        for item in tbar:
            x = item[0].to(device).long()
            question_id = item[1].to(device).long()
            chapter = item[2].to(device).long()
            schapter = item[3].to(device).long()
            responses = item[4].to(device).long()
            user_feats = item[5].to(device).float()
            label = item[6].to(device).float()            
            target_mask = (question_id!=0)
            with torch.no_grad():
                output = model(x, question_id, chapter, schapter, responses, user_feats)

            output = torch.reshape(output, label.shape)
            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            train_loss.append(loss.item())

            pred = (torch.sigmoid(output) >= 0.5).long()
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('valid loss - {:.4f}'.format(loss))

        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.average(train_loss)

        return loss, acc, auc
    
    MIN_VAL = 1000000000
    count = 0
    print('----------------------------------------------------------------------------')
    for epoch in range(epochs):
        train_loss, train_acc, train_auc = train_epoch(model=saint, device=device)
        print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, train_loss, train_acc, train_auc))
        val_loss, val_acc, val_auc = val_epoch(model=saint, val_iterator= val_dataloader, device=device)
        print("epoch - {} val_loss - {:.2f} val acc - {:.3f} val auc - {:.3f}".format(epoch, val_loss, val_acc, val_auc))
        if val_loss < MIN_VAL:
            count = 0
            MIN_VAL = val_loss
        else:
            count += 1
        if count == patience:
            print('Val Loss does not improve for {} consecutive epochs'.format(patience))
            break
    test_loss, test_acc, test_auc = val_epoch(model=saint, device=device)
    print("epoch - {} test_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, test_loss, test_acc, test_auc))
    train_loss, train_acc, train_auc = val_epoch(model=saint, val_iterator=train_dataloader, device=device)

    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    test_accs.append(test_acc)
    train_aucs.append(train_auc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

  0%|          | 0/14 [00:00<?, ?it/s]

----------------------------------------------------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 0 train_loss - 0.59 acc - 0.698 auc - 0.619


  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 28 val_loss - 0.51 val acc - 0.737 val auc - 0.802
Val Loss does not improve for 5 consecutive epochs


  0%|          | 0/14 [00:00<?, ?it/s]

epoch - 28 test_loss - 0.36 acc - 0.828 auc - 0.874


valid loss - 0.4099: 100%|██████████| 14/14 [00:05<00:00,  2.47it/s]


In [31]:
#display test loss/acc/auc
print("test avg loss: ", np.mean(test_losses), np.std(test_losses) )
print("test avg acc: ", np.mean(test_accs), np.std(test_accs))
print("test avg auc: ", np.mean(test_aucs), np.std(test_aucs))

test avg loss:  0.35347802340984347 0.009303784151227944
test avg acc:  0.8330877798197903 0.007046767619630736
test avg auc:  0.8793755424291371 0.0037583937137915854


In [32]:
#display train loss/acc/auc
print("train avg loss: ", np.mean(train_losses), np.std(train_losses) )
print("train avg acc: ", np.mean(train_accs), np.std(train_accs))
print("train avg auc: ", np.mean(train_aucs), np.std(train_aucs))

train avg loss:  0.39572287883077345 0.007946595976225667
train avg acc:  0.8079092583638436 0.004709178094335251
train avg auc:  0.8727422896939796 0.005833593484179244
